In [2]:
import pandas as pd

In [3]:
file_path = '../../dataSource/phenos.xlsx'
df = pd.read_excel(file_path,header=4)
df.head()

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,Treatment,Replicate,Entry,Block,Plot number,Field row,Field column,Days till flowering (50%) (DTF),Chlorophyll content index (CCI),Leaf area index (%) (LAI),"Flowering success (flowering and filled grain production occurred, yes or no)",Fecundity (Filled grain number)
0,Wet,1,e40,1,1,1,1,73.0,7.8,0.292157,1,180.0
1,Dry,1,e40,1,1,1,1,71.0,17.2,0.153312,1,16.0
2,Wet,1,e167,1,2,2,1,68.0,7.5,4.482489,1,1010.0
3,Dry,1,e167,1,2,2,1,71.0,7.3,0.256249,1,139.0
4,Wet,1,e115,1,3,3,1,57.0,8.9,4.415642,1,961.0


In [4]:
df = df.rename(columns={
    'Days till flowering (50%) (DTF)': 'DTF',
    'Chlorophyll content index (CCI)': 'CCI',
    'Flowering success (flowering and filled grain production occurred, yes or no)': 'FS',
    'Fecundity (Filled grain number)': 'Fecundity',
    'Leaf area index (%) (LAI)': 'LAI',
})
df

,Treatment,Replicate,Entry,Block,Plot number,Field row,Field column,DTF,CCI,LAI,FS,Fecundity
0,Wet,1,e40,1,1,1,1,73.0,7.8,0.292157,1,180.0
1,Dry,1,e40,1,1,1,1,71.0,17.2,0.153312,1,16.0
2,Wet,1,e167,1,2,2,1,68.0,7.5,4.482489,1,1010.0
3,Dry,1,e167,1,2,2,1,71.0,7.3,0.256249,1,139.0
4,Wet,1,e115,1,3,3,1,57.0,8.9,4.415642,1,961.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1315,Dry,3,e143,22,658,218,3,NaN,15.5,NaN,1,9.0
1316,Wet,3,e116,22,659,219,3,63.0,26.1,0.415854,1,302.0
1317,Dry,3,e116,22,659,219,3,65.0,26.7,NaN,1,13.0
1318,Wet,3,e4,22,660,220,3,77.0,18.9,0.988180,1,530.0


In [5]:
df.to_csv('../results/phenos_rename.csv', index=False)

In [8]:
df = pd.read_csv('../results/phenos_rename.csv')

In [9]:
df['EntryNumber'] = df['Entry'].str[1:].astype(int)
df2 = df.sort_values(by=['EntryNumber','Treatment','Replicate'])
df2['Entry_Treatment_Replicate'] = df2['Entry'].astype(str) + '_'+df2['Treatment'] + '_'  + df2['Replicate'].astype(str)
first_col = 'Entry_Treatment_Replicate'
df2 = pd.concat([df2[first_col], df2.drop(columns=[first_col])], axis=1)
df2 = df2.drop(columns=['Block','Plot number','Field row','Field column'])
df2.to_csv('../results/phenos_rename_sorted.csv', index=False)

In [11]:
# 数据处理规则: 每三行为一组，如果一组数据中缺失值数量为1个，取没有缺失的值填补缺失值，如果缺失值数量为2个，取没有缺失的值的平均值填补缺失值，如果缺失值数量为3个，则用0填补缺失值
def fill_missing_values(group):
    # 计算缺失值的数量
    missing_count = group.isnull().sum()
    # 计算没有缺失值的数量
    non_missing_count = len(group) - missing_count
    # 如果没有缺失值，直接返回原组
    if non_missing_count == len(group):
        return group
    # 如果有一个缺失值，取没有缺失的值填补缺失值
    elif missing_count == 1:
        return group.fillna(group.dropna().mean())
    # 如果有两个缺失值，取没有缺失的值的平均值填补缺失值
    elif missing_count == 2:
        return group.fillna(group.dropna().mean())
    # 如果有三个缺失值，则用0填补缺失值
    else:
        return group.fillna(0)
# 读取数据
df = pd.read_csv('../results/phenos_rename_sorted.csv')
# 按照每三行分组
grouped = df.groupby(df.index // 3)
# 对每组数据应用填补缺失值的函数
df_filled = grouped.apply(lambda x: x.apply(fill_missing_values, axis=0))
df_filled
# # 重置索引
df_filled = df_filled.reset_index(drop=True)
# # 保存填补后的数据  
df_filled.to_csv('../results/phenos_rename_sorted_filled.csv', index=False)
# # # 读取数据  
# df_filled = pd.read_csv('./processed_data/phenos_rename_sorted_filled.csv')